In [ ]:
import sys
sys.dont_write_bytecode = True
import scipy as sp
from scipy import io as sio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scienceplots
%config InlineBackend.figure_format = 'svg'
# plt.style.use(['science', 'notebook','grid'])

matlab_files = ['./data/matlab/TypeA_field_N40high.mat',
                './data/matlab/TypeA_field_N40low.mat']

femm_files_0A = ['./data/femm/TypeA_field_10mm_0A_N40low.npz', 
                './data/femm/TypeA_field_10mm_0A_N40high.npz',
                './data/femm/TypeA_field_10mm_0A_N40initial.npz',]
femm_files_1A = ['./data/femm/TypeA_field_10mm_1A_N40low.npz',
                './data/femm/TypeA_field_10mm_1A_N40high.npz',
                './data/femm/TypeA_field_10mm_1A_N40initial.npz',]

## Matlab

In [ ]:
for i, file in enumerate(matlab_files):
    data = sio.loadmat(file)
    plt.figure(figsize=(8,6))
    plt.subplot(1,2,1)
    plt.contour(data['X1']*1000, data['Y1']*1000,data['Bmag1'], levels=20, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.subplot(1,2,2)
    plt.contour(data['X2']*1000, data['Y2']*1000,data['Bmag2'], levels=20, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.suptitle('Bxmag_matlab + ' + file.split('/')[-1])
    plt.tight_layout()
    # plt.savefig('./data/plot/pdf_files/Bmag_matlab + ' + file.split('field_')[-1].split('.mat')[0] + '_at_center.pdf',format='pdf',dpi=300)
    # plt.savefig('./data/plot/svg_files/Bmag_matlab + ' + file.split('field_')[-1].split('.mat')[0] + '_at_center.svg',format='svg',dpi=300)


## FEMM

In [ ]:
def femm_analyser(data, keys, num=0, reverse = False):
    """ 
    Function to analyse the data from the FEMM simulation
    data: list of files
    keys: list of keys to identify the data
    num: index of the position to extract the field information, for a dataset with 11 points, chose 5 to be in the center.
    reverse: boolean to reverse the sign of half of the data.
    """
    fkey = keys

    force_dict = {key: {'positions': [], 'force': []} for key in fkey}
    
    field_dict = {key:{'coil_pos': None, 'upp_xpos': [], 'upp_ypos': [], 'low_xpos': [], 'low_ypos': [],
                       'upp_Bx'  : [], 'upp_By'  : [],  'low_Bx'  : [], 'low_By'  : [], 
                       'upp_Bmag': [],'low_Bmag': []} for key in fkey}
    
    for f, file in enumerate(data):
        dat = np.load(file, allow_pickle=True)
        upp_data = dat['Magnetic_field_upper']
        low_data = dat['Magnetic_field_lower']

        pos = np.real(dat['IC_positions'])    
        idx = np.arange(0, len(pos))
        
        if reverse:
            pos = [-x if i < len(pos) // 2 else x for i, x in enumerate(pos)]
        
        force_dict[fkey[f]]['positions'] = pos
        force_dict[fkey[f]]['force'] = np.zeros(len(idx))

        for id in idx:
            upp_data_arr = np.array([x for x in upp_data[id]])
            lwo_data_arr = np.array([y for y in low_data[id]])

            upp_xpos = upp_data_arr[:, 0].reshape(7,48).T
            upp_ypos = upp_data_arr[:, 1].reshape(7,48).T
            low_xpos = lwo_data_arr[:, 0].reshape(7,48).T
            low_ypos = lwo_data_arr[:, 1].reshape(7,48).T

            field_dict[fkey[f]]['upp_xpos'] = upp_xpos
            field_dict[fkey[f]]['upp_ypos'] = upp_ypos
            field_dict[fkey[f]]['low_xpos'] = low_xpos
            field_dict[fkey[f]]['low_ypos'] = low_ypos
    
            upp_field = upp_data_arr[:, 2]
            low_field = lwo_data_arr[:, 2]

            upp_Bx = np.array([x[0] for x in upp_field]).reshape(7,48).T
            upp_By = np.array([x[1] for x in upp_field]).reshape(7,48).T
            low_Bx = np.array([x[0] for x in low_field]).reshape(7,48).T
            low_By = np.array([x[1] for x in low_field]).reshape(7,48).T

            upp_Bmag = np.array([np.sqrt(np.sum(np.array(x)**2)) for x in upp_field]).reshape(7,48).T
            low_Bmag = np.array([np.sqrt(np.sum(np.array(x)**2)) for x in low_field]).reshape(7,48).T 
            
            upp_Force = 0
            low_Force = 0
            
            for col in upp_Bx:
                for row in col:
                    upp_Force +=  row * 2 * np.pi * 0.035
            for col in low_Bx:
                for row in col:
                    low_Force +=  row * 2 * np.pi * 0.035

            force_dict[fkey[f]]['force'][id] = abs(upp_Force - low_Force) * 1000
            
            if id == num:
            
                field_dict[fkey[f]]['coil_pos'] = pos[num]
                field_dict[fkey[f]]['upp_Bx'] = upp_Bx
                field_dict[fkey[f]]['upp_By'] = upp_By
                field_dict[fkey[f]]['low_Bx'] = low_Bx
                field_dict[fkey[f]]['low_By'] = low_By
                field_dict[fkey[f]]['upp_Bmag'] = upp_Bmag
                field_dict[fkey[f]]['low_Bmag'] = low_Bmag
    return force_dict, field_dict

In [ ]:
result_0A = femm_analyser(femm_files_0A, ['0A_N40low','0A_N40high','0A_N40initial'],num = 5, reverse = True)
result_1A = femm_analyser(femm_files_1A, ['1A_N40low','1A_N40high','1A_N40initial'],num = 5, reverse = True)

### Force

In [ ]:
plt.figure(figsize=(8,6))     
plt.style.use(['science', 'notebook','grid'])  
for key in result_0A[0].keys():
    plt.plot(result_0A[0][key]['positions'], result_0A[0][key]['force'],'o-', label=key, markersize = 10)
    plt.xlabel('Position [mm]')
    plt.ylabel('Force [mN]')
    plt.legend()
    plt.grid()
    plt.title('Force vs Position')
for key in result_1A[0].keys():
    plt.plot(result_1A[0][key]['positions'], result_1A[0][key]['force'],'x-', label=key,markersize = 10)
    plt.xlabel('Position [mm]')
    plt.ylabel('Force [mN]')
    plt.legend(fontsize = 12, loc = 'lower center')
    plt.grid()
    plt.title('Force vs Position')
plt.ylim(500,670)
# plt.xlim(-10,10)
plt.tight_layout()
# plt.savefig('./data/plot/pdf_files/Force_vs_Position_N40_Comparision.pdf',format='pdf',dpi=300)
# plt.savefig('./data/plot/svg_files/Force_vs_Position_N40_Comparision.svg',format='svg',dpi=300)

### B magnitude

In [ ]:
for key in result_0A[1].keys():
    plt.figure(figsize=(8,6))
    plt.subplot(1,2,1)
    plt.contour(result_0A[1][key]['upp_xpos'], result_0A[1][key]['upp_ypos'], result_0A[1][key]['upp_Bmag'], levels=25, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.subplot(1,2,2)
    plt.contour(result_0A[1][key]['low_xpos'], result_0A[1][key]['low_ypos'], result_0A[1][key]['low_Bmag'], levels = 25)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.suptitle('Bmag_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm')
    plt.colorbar(label='B Field (T)')
    plt.tight_layout()
    # plt.savefig('./data/plot/pdf_files/'+ 'Bmag_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm' +'.pdf',format='pdf',dpi=300)
    # plt.savefig('./data/plot/svg_files/'+ 'Bmag_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm' +'.svg',format='svg',dpi=300)
for key in result_1A[1].keys():
    plt.figure(figsize=(8,6))
    plt.subplot(1,2,1)
    plt.contour(result_1A[1][key]['upp_xpos'], result_1A[1][key]['upp_ypos'], result_1A[1][key]['upp_Bmag'], levels=25, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.subplot(1,2,2)
    plt.contour(result_1A[1][key]['low_xpos'], result_1A[1][key]['low_ypos'], result_1A[1][key]['low_Bmag'], levels = 25)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.suptitle('Bmag_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm')
    plt.colorbar(label='B Field (T)')
    plt.tight_layout()
    # plt.savefig('./data/plot/pdf_files/'+'Bmag_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm'+'.pdf',format='pdf',dpi=300)
    # plt.savefig('./data/plot/svg_files/'+'Bmag_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm'+'.svg',format='svg',dpi=300)

### Bx magnitude

In [ ]:
for key in result_0A[1].keys():
    plt.figure(figsize=(8,6))
    plt.subplot(1,2,1)
    plt.contour(result_0A[1][key]['upp_xpos'], result_0A[1][key]['upp_ypos'], result_0A[1][key]['upp_Bx'], levels=25, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.subplot(1,2,2)
    plt.contour(result_0A[1][key]['low_xpos'], result_0A[1][key]['low_ypos'], result_0A[1][key]['low_Bx'], levels = 25)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.suptitle('Bx_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm')
    plt.colorbar(label='B Field (T)')
    plt.tight_layout()
    # plt.savefig('./data/plot/pdf_files/'+ 'Bx_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm' +'.pdf',format='pdf',dpi=300)
    # plt.savefig('./data/plot/svg_files/'+ 'Bx_femm + '+key+' at ' + str(result_0A[1][key]['coil_pos']) +' mm' +'.svg',format='svg',dpi=300)
for key in result_1A[1].keys():
    plt.figure(figsize=(8,6))
    plt.subplot(1,2,1)
    plt.contour(result_1A[1][key]['upp_xpos'], result_1A[1][key]['upp_ypos'], result_1A[1][key]['upp_Bx'], levels=25, cmap='viridis')
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.colorbar(label='B Field (T)')
    plt.subplot(1,2,2)
    plt.contour(result_1A[1][key]['low_xpos'], result_1A[1][key]['low_ypos'], result_1A[1][key]['low_Bx'], levels = 25)
    plt.xlabel('X')
    plt.ylabel('Y')
    plt.suptitle('Bx_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm')
    plt.colorbar(label='B Field (T)')
    plt.tight_layout()
    # plt.savefig('./data/plot/pdf_files/'+'Bx_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm'+'.pdf',format='pdf',dpi=300)
    # plt.savefig('./data/plot/svg_files/'+'Bx_femm + '+key+' at ' + str(result_1A[1][key]['coil_pos']) +' mm'+'.svg',format='svg',dpi=300)